In [1]:
from eia_client import Client

In [2]:
client = Client()

In [3]:
# Dataset Info includes all non-leaf-node data series (i.e. parents of other datasets or of series)
electricity = client.dataset_info("electricity")
electricity.routes

[RouteInfo(id='retail-sales', name='Electricity Sales to Ultimate Customers', description='Electricity sales to ultimate customer by state and sector (number of customers, average price, revenue, and megawatthours of sales).  \n    Sources: Forms EIA-826, EIA-861, EIA-861M'),
 RouteInfo(id='electric-power-operational-data', name='Electric Power Operations (Annual and Monthly)', description='Monthly and annual electric power operations by state, sector, and energy source.\n    Source: Form EIA-923'),
 RouteInfo(id='rto', name='Electric Power Operations (Daily and Hourly)', description='Hourly and daily electric power operations by balancing authority.  \n    Source: Form EIA-930'),
 RouteInfo(id='state-electricity-profiles', name='State Specific Data', description='State Specific Data'),
 RouteInfo(id='operating-generator-capacity', name='Inventory of Operable Generators', description='Inventory of operable generators in the U.S.\n    Source: Forms EIA-860, EIA-860M'),
 RouteInfo(id='fa

In [4]:
# Series Info includes data series with specific data elements and facets
operational = client.series_info("electricity/electric-power-operational-data")
operational.data

{'generation': DataInfo(alias='Utility Scale Electricity Net Generation', units=None),
 'total-consumption': DataInfo(alias='Consumption of Fuels for Electricity Generation and Useful Thermal Output (Physical Units)', units=None),
 'consumption-for-eg': DataInfo(alias='Consumption of Fuels for Electricity Generation (Physical Units)', units=None),
 'consumption-uto': DataInfo(alias='Consumption of Fuels for Useful Thermal Output (Physical Units)', units=None),
 'total-consumption-btu': DataInfo(alias='Consumption of Fuels for Electricity Generation and Useful Thermal Output (BTUs)', units=None),
 'consumption-for-eg-btu': DataInfo(alias='Consumption of Fuels for Electricity Generation (BTUs)', units=None),
 'consumption-uto-btu': DataInfo(alias='Consumption of Fuels for Useful Thermal Output (BTUs)', units=None),
 'stocks': DataInfo(alias='Stocks of Fuel (Physical Units)', units=None),
 'receipts': DataInfo(alias='Receipts of Fuel (Physical Units)', units=None),
 'receipts-btu': DataIn

In [5]:
# The response is cached, so the second call is much faster
operational = client.series_info("electricity/electric-power-operational-data")
operational.facets

[FacetInfo(id='location', description='State / Census Region'),
 FacetInfo(id='sectorid', description='Sector'),
 FacetInfo(id='fueltypeid', description='Energy Source')]

In [6]:
# Getting data returns a dataframe object
df = client.get("electricity/electric-power-operational-data", data=["generation", "total-consumption"], facets={"fueltypeid": ["SUB"]})

Fetching data for electricity/electric-power-operational-data with offset 0
Fetching data for electricity/electric-power-operational-data with offset 5000
Fetching data for electricity/electric-power-operational-data with offset 10000
Fetching data for electricity/electric-power-operational-data with offset 15000
Fetching data for electricity/electric-power-operational-data with offset 20000
Fetching data for electricity/electric-power-operational-data with offset 25000
Fetching data for electricity/electric-power-operational-data with offset 30000
Fetching data for electricity/electric-power-operational-data with offset 35000
Fetching data for electricity/electric-power-operational-data with offset 40000
Fetching data for electricity/electric-power-operational-data with offset 45000
Fetching data for electricity/electric-power-operational-data with offset 50000
Fetching data for electricity/electric-power-operational-data with offset 55000
Fetching data for electricity/electric-power-

In [8]:
df

,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units,total-consumption,total-consumption-units
0,2022-12,OH,Ohio,99,All Sectors,SUB,subbituminous coal,38.14917,thousand megawatthours,16.448,thousand short tons
1,2022-11,MO,Missouri,1,Electric Utility,SUB,subbituminous coal,2392.41242,thousand megawatthours,1409.334,thousand short tons
2,2022-11,KY,Kentucky,98,Electric Power,SUB,subbituminous coal,729.52632,thousand megawatthours,449.992,thousand short tons
3,2021-08,ENC,East North Central,1,Electric Utility,SUB,subbituminous coal,2888.14841,thousand megawatthours,1792.265,thousand short tons
4,2021-08,ENC,East North Central,2,IPP Non-CHP,SUB,subbituminous coal,1923.01959,thousand megawatthours,1225.726,thousand short tons
...,...,...,...,...,...,...,...,...,...,...,...
4168,2021-10,MT,Montana,2,IPP Non-CHP,SUB,subbituminous coal,1016.50613,thousand megawatthours,630.493,thousand short tons
4169,2021-02,WY,Wyoming,1,Electric Utility,SUB,subbituminous coal,1431.71467,thousand megawatthours,900.004,thousand short tons
4170,2022-12,SD,South Dakota,1,Electric Utility,SUB,subbituminous coal,0.00000,thousand megawatthours,0.158,thousand short tons
4171,2021-02,IL,Illinois,2,IPP Non-CHP,SUB,subbituminous coal,1064.28639,thousand megawatthours,683.059,thousand short tons


In [19]:
df2 = client.get("electricity/electric-power-operational-data", data=["generation", "total-consumption"], facets={"fueltypeid": ["SUB"], "location": ["NC"], "sector": ["99"]})

Fetching data for electricity/electric-power-operational-data with offset 0


KeyError: 'sector'

In [10]:
df2

,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units,total-consumption,total-consumption-units
0,2005-05,NC,North Carolina,99,All Sectors,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
1,2005-05,NC,North Carolina,90,Electric Power Sector Non-CHP,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
2,2005-06,NC,North Carolina,98,Electric Power,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
3,2005-06,NC,North Carolina,90,Electric Power Sector Non-CHP,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
4,2005-05,NC,North Carolina,98,Electric Power,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
5,2005-05,NC,North Carolina,1,Electric Utility,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
6,2005-06,NC,North Carolina,99,All Sectors,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
7,2005-06,NC,North Carolina,1,Electric Utility,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons


In [13]:
df[df["location"] == "NC"].equals(df2)

False

In [18]:
df[df["location"] == "NC"].reset_index().equals(df2)

False

In [15]:
len(df2)

8